In [10]:
import os
import sys
import tensorflow as tf
import keras_tuner as kt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import trange, tqdm
from pickle import (
    dump,
    load
)
from tensorflow.keras import (
    layers,
    Model,
    Sequential,
    optimizers
)
from pprint import pprint
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler



In [11]:
BASE_DIR = "./"
DATA_FOLDER_NAME = "data"
DATA_FOLDER_PATH = os.path.join(BASE_DIR, DATA_FOLDER_NAME)
MODELS_FOLDER_NAME = "models"
MODELS_FOLDER_PATH = os.path.join(BASE_DIR, MODELS_FOLDER_NAME)
TUNERS_FOLDER_NAME = "tuners"
TUNERS_FOLDER_PATH = os.path.join(BASE_DIR, TUNERS_FOLDER_NAME)


OUTPUT_DATA_COLUMNS = ["position_x", "position_y", "position_z"]
INPUT_DATA_COLUMNS = ["angular_acceleration_x", "angular_acceleration_y", "angular_acceleration_z",
                      "angular_velocity_x", "angular_velocity_y", "angular_velocity_z",
                      "linear_acceleration_x", "linear_acceleration_y", "linear_acceleration_z",
                      "linear_velocity_x", "linear_velocity_y", "linear_velocity_z",
                      "orientation_x", "orientation_y", "orientation_z", "orientation_w", "motor_state_timestamp",
                      "barometer_altitude", "barometer_pressure", "barometer_qnh", "barometer_timestamp",
                      "magnetometer_magnetic_field_body_x", "magnetometer_magnetic_field_body_y",
                      "magnetometer_magnetic_field_body_x", "magnetometer_timestamp",
                      "rotor_a_speed", "rotor_a_thrust", "rotor_a_torque_scaler",
                      "rotor_b_speed", "rotor_b_thrust", "rotor_b_torque_scaler",
                      "rotor_c_speed", "rotor_c_thrust", "rotor_c_torque_scaler",
                      "rotor_d_speed", "rotor_d_thrust", "rotor_d_torque_scaler",
                      "rotor_timestamp"
                     ]

TIMESTAMP_COLUMNS = [
    "motor_state_timestamp",
    "barometer_timestamp",
    "magnetometer_timestamp",
    "rotor_timestamp"
]


INPUT_SEQUENCE_COLUMNS = ["angular_acceleration_x", "angular_acceleration_y", "angular_acceleration_z",
                          "linear_acceleration_x", "linear_acceleration_y", "linear_acceleration_z",
                          "orientation_x", "orientation_y", "orientation_z", "orientation_w", "motor_state_timestamp",
                          "barometer_altitude", "barometer_pressure", "barometer_qnh", "barometer_timestamp",
                          "magnetometer_magnetic_field_body_x", "magnetometer_magnetic_field_body_y",
                          "magnetometer_magnetic_field_body_x", "magnetometer_timestamp",
                          "rotor_a_speed", "rotor_a_thrust", "rotor_a_torque_scaler",
                          "rotor_b_speed", "rotor_b_thrust", "rotor_b_torque_scaler",
                          "rotor_c_speed", "rotor_c_thrust", "rotor_c_torque_scaler",
                          "rotor_d_speed", "rotor_d_thrust", "rotor_d_torque_scaler",
                          "rotor_timestamp"
                         ]
OUTPUT_SEQUENCE_COLUMNS = ["position_x", "position_y", "position_z"]
MAIN_TIMESTAMP_COLUMN = "motor_state_timestamp"
INPUT_SEQUENCE_LENGTH = 100


In [12]:
def save_model_with_scalers_binary(model, scaler_x, scaler_y, model_name: str):
    """
    Saves models with the x, y scaler objects to a binary library using pickle library
    """
    model_file_name = f"{model_name}_model.pkl"
    model_file_path = os.path.join(MODELS_FOLDER_PATH, model_file_name)
    scaler_x_file_name = f"{model_name}_scaler_x.pkl"
    scaler_x_file_path = os.path.join(MODELS_FOLDER_PATH, scaler_x_file_name)
    scaler_y_file_name = f"{model_name}_scaler_y.pkl"
    scaler_y_file_path = os.path.join(MODELS_FOLDER_PATH, scaler_y_file_name)

    with open(model_file_path, "wb") as file:
        dump(model, file)

    with open(scaler_x_file_path, "wb") as file:
        dump(scaler_x, file)

    with open(scaler_y_file_path, "wb") as file:
        dump(scaler_y, file)
        
def load_model_with_scalers_binary(model_name: str):
    """
    Saves models with the x, y scaler objects to a binary library using pickle library
    """
    model_file_name = f"{model_name}_model.pkl"
    model_file_path = os.path.join(MODELS_FOLDER_PATH, model_file_name)
    scaler_x_file_name = f"{model_name}_scaler_x.pkl"
    scaler_x_file_path = os.path.join(MODELS_FOLDER_PATH, scaler_x_file_name)
    scaler_y_file_name = f"{model_name}_scaler_y.pkl"
    scaler_y_file_path = os.path.join(MODELS_FOLDER_PATH, scaler_y_file_name)

    with open(model_file_path, "rb") as file:
        model = load(file)

    with open(scaler_x_file_path, "rb") as file:
        scaler_x = load(file)

    with open(scaler_y_file_path, "rb") as file:
        scaler_y = load(file)

    return model, scaler_x, scaler_y

In [13]:
def split_data(data: np.array):
    """
    Splits data into train, dev and test
    :return:
    """
    data_len = len(data)

    train, dev, test = np.split(data, [int(.7 * data_len), int(.95 * data_len)])

    return train, dev, test


def _convert_timestamp_to_interval_seconds(flight_input_df: pd.DataFrame, timestamp_columns: list):
    """
    Converts the timestamp fields into the amount of seconds between each two timestamps

    Note: each timestamp represents the amount eof NANO seconds (1,000,000,000 nanoseconds = 1 seconds)
    """
    # Converts the start time to time interval
    next_time_df = flight_input_df[timestamp_columns].shift(-1)
    time_diff_df = (next_time_df - flight_input_df[timestamp_columns]) / 1_000_000_000
    flight_input_df.loc[:, timestamp_columns] = time_diff_df
    return flight_input_df


def _convert_location_to_step(flight_output_df: pd.DataFrame):
    next_coordinates_df = flight_output_df.shift(-1)
    coordinate_diff = flight_output_df - next_coordinates_df

    return coordinate_diff


def load_flight_steps_from_file(csv_name: str, input_columns: list, output_columns: list):
    """

    @param csv_name:
    @param input_columns:
    @param output_columns:
    @return:
    """
    if not csv_name.endswith("csv"):
        raise ValueError(f"File with unsupported extension, expected csv (file: {csv_name})")

    csv_path = os.path.join(DATA_FOLDER_PATH, csv_name)
    flight_df = pd.read_csv(csv_path)

    x_df = flight_df[input_columns].copy()
    timestamp_columns = [column for column in input_columns if column in TIMESTAMP_COLUMNS]
    x_df = _convert_timestamp_to_interval_seconds(x_df, timestamp_columns)

    y_df = flight_df[output_columns].copy()
    y_df = _convert_location_to_step(y_df)

    # Drops the last record because the process is based of difference
    x_df.drop(x_df.tail(1).index, inplace=True)
    y_df.drop(y_df.tail(1).index, inplace=True)

    return x_df, y_df

In [34]:
model_name = "s2s_50seq_1"
data_csv_name = "_flight_2021:12:31_21:56:52_record.csv"
input_columns = INPUT_SEQUENCE_COLUMNS
output_columns = OUTPUT_SEQUENCE_COLUMNS
sequence_length = INPUT_SEQUENCE_LENGTH

flight_x_df, flight_y_df = load_flight_steps_from_file(data_csv_name, input_columns, output_columns)
data_x = flight_x_df.to_numpy()
real_y = flight_y_df.to_numpy()

try:
    model, scaler_x, scaler_y = load_model_with_scalers_binary(model_name)
except FileNotFoundError:
    print(f"There is no model in name: {model_name}")


In [35]:
normalized_real_x = scaler_x.transform(data_x)
recording_length = data_x.shape[0]

# Splits the data into data sequences
sequences_x = []
for offset in range(recording_length - sequence_length):
    sequences_x.append(normalized_real_x[offset: offset + sequence_length, :])

sequences_x = np.stack(sequences_x)
predicted_sequence = model.predict(sequences_x)
predicted_sequence_len = predicted_sequence.shape[0]

In [40]:
# dir(model)
lstm_layer = model.layers[1]
print(lstm_layer.states)
print(lstm_layer.return_state)
print(lstm_layer.units)

ListWrapper([None, None])
False
32


In [ ]:

    return




# opposite operation of np.add.accumulate
for pred_index in range(predicted_sequence_len):
    for seq_index in range(sequence_length - 1, 0 ,-1) :
        predicted_sequence[pred_index][seq_index] -= predicted_sequence[pred_index][seq_index - 1]

predicted_values = np.zeros(real_y.shape)

# finds best betta
betta = 0.0
best_betta = 0.0
normalized_real_y = scaler_y.transform(real_y)
lowerst_mse = sys.float_info.max
while betta < 1:
    predicted_values = np.zeros(real_y.shape)
    values_in_average = np.zeros(real_y.shape)
    for seq_index in range(predicted_sequence_len):
        values_in_average[seq_index:seq_index + sequence_length] += 1
        predicted_values[seq_index:seq_index + sequence_length] = \
            (betta * predicted_values[seq_index:seq_index + sequence_length] + 
            (1 - betta) * predicted_sequence[seq_index, :]) 
#             / \
#             (1 - np.power(betta, values_in_average[seq_index:seq_index + sequence_length]))

    mse = ((predicted_values - normalized_real_y)**2).mean(axis=0).reshape((3,1))
    if not np.isnan(np.sum(mse)) and np.sum(np.sqrt(np.power(mse, 2))) < np.sum(np.sqrt(np.power(lowerst_mse,2))):
#             if not np.isnan(np.sum(mse)) and np.sum(mse) < np.sum(lowerst_mse):
        lowerst_mse = mse
        best_betta = betta
        print(mse)
    print(betta)
    betta += 0.01

# Inserts all the predicted values using Exponentially Weighted Averages with bias correction
values_in_average = np.zeros(real_y.shape)
#     betta = 0.94
betta = best_betta
for seq_index in range(predicted_sequence_len):
    values_in_average[seq_index:seq_index + sequence_length] += 1
    predicted_values[seq_index:seq_index + sequence_length] = \
        (betta * predicted_values[seq_index:seq_index + sequence_length] + 
        (1 - betta) * predicted_sequence[seq_index, :]) 


#     normalized_real_y = scaler_y.transform(real_y)
#     print(predicted_values[100:110])
#     print(normalized_real_y[100:110])

#     # Inserts all the predicted values using Exponentially Weighted Averages with bias correction
#     values_in_average = np.zeros(real_y.shape)
#     for seq_index in range(predicted_sequence_len):
#         values_in_average[seq_index:seq_index + sequence_length] += 1
#         predicted_values[seq_index:seq_index + sequence_length] += predicted_sequence[seq_index,:]
#     predicted_values = predicted_values /  values_in_average 

predicted_values = scaler_y.inverse_transform(predicted_values)

print(real_y[100:110])
print(predicted_values[100:110])

predicted_offset = np.add.accumulate(predicted_values)
real_offset = np.add.accumulate(real_y)

print(predicted_offset[100:120])
print(real_offset[100:120])

#     print(real_offset)
time_intervals = flight_x_df[MAIN_TIMESTAMP_COLUMN].to_numpy().reshape(-1, 1)
time_offset = np.add.accumulate(time_intervals)

return predicted_offset, real_offset
